In [2]:
import pandas as pd
import csv
from collections import defaultdict
import sqlite3

class Caption:
    def __init__(self, items):
        self.idx = items[0]
        self.file_id = items[1]
        self.filename = self.file_id.replace(".jpg", ".txt")
        self.caption = items[2].strip().strip(".").strip()
        self.author = items[3]
        self.is_occluded = items[4]
        self.is_error = items[5]

    def __str__(self):
        return "id:{},file_id:{},filename:{},caption:{},user:{},is_occleded:{},is_error:{}".format(
            self.idx, self.file_id, self.filename, 
            self.caption, self.author, self.is_occluded, self.is_error
        )

In [3]:
dicta = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0)))
with open("caption.csv" , "r") as fp:
    csvfp = csv.reader(fp)
    for v in csvfp:
        cap = Caption(v)
        dicta[cap.file_id][cap.author][cap.caption] += 1
for file_id, user_cap_count in dicta.items():
    for user, caption_count in user_cap_count.items():
        total = 0
        for caption, count in caption_count.items():
            total += count
            if count > 1:
                print("DUPLICATE:", file_id, user, caption, count)
        if total > 2:
            print("MORE 2:", file_id, user, caption)
    

DUPLICATE: P00004.jpg tejas Two knives intersecting each other on left side of bag 2
DUPLICATE: P00008.jpg soumen Two knives are hidden inside a luggage 2
MORE 2: P00008.jpg soumen At the bottom of a trolley bag, two guns and a few knives are hidden
DUPLICATE: P00010.jpg soumen Two knives are hidden inside a luggage scattered around 2
MORE 2: P00010.jpg soumen Two knives are secretly tucked away inside a luggage that is scattered around
DUPLICATE: P00013.jpg soumen a luggage is hiding a few knives 2
MORE 2: P00013.jpg soumen At the bottom of a trolley bag, two guns and two knives are concealed
DUPLICATE: P00026.jpg soumen Two sharp knives and a few water bottles in a backpack 2
MORE 2: P00026.jpg soumen Two sharp knives and several water bottles are inside a backpack
DUPLICATE: P00027.jpg soumen Two sharp knives and a few water bottles in a backpack 2
MORE 2: P00027.jpg soumen Two sharp knives and several water bottles are inside a backpack
DUPLICATE: P00028.jpg soumen Two sharp knives

# Read form SQLite

In [4]:
conn = sqlite3.connect("tip_gai_20230522_2136.db")
curr = conn.cursor()
df = pd.read_sql_query("SELECT * FROM caption", conn)
df.head()

,id,file_id,caption,user,is_error,is_occluded
0,1,P00001.jpg,"Two knives, one placed on top of other, in a b...",anshul,0,1
1,2,P00001.jpg,A bag with knives.\nTwo shrap knives in a back...,soumen,0,1
2,3,P00001.jpg,Security discovered a concealed knife in the p...,soumen,0,1
3,5,P00004.jpg,Two knives are hidden inside a backpack.,soumen,0,0
4,6,P00005.jpg,Two knives are hidden inside a backpack overla...,soumen,0,0


In [11]:
#!pip install language-tool-python
import language_tool_python
def error_check_and_correct(text, correction=False):
    # using the tool
    my_tool = language_tool_python.LanguageTool('en-US')

    # getting the matches
    my_matches = my_tool.check(text)

    # defining some variables
    myMistakes = []
    if correction:
        myCorrections = []
    startPositions = []
    endPositions = []

    # using the for-loop
    for rules in my_matches:
        if len(rules.replacements) > 0:
            startPositions.append(rules.offset)
            endPositions.append(rules.errorLength + rules.offset)
            myMistakes.append(my_text[rules.offset : rules.errorLength + rules.offset])
            if correction:
                myCorrections.append(rules.replacements[0])
    
    if correction:
        # creating new object
        my_NewText = list(my_text)

        # rewriting the correct passage
        for n in range(len(startPositions)):
            for i in range(len(my_text)):
                my_NewText[startPositions[n]] = myCorrections[n]
                if (i > startPositions[n] and i < endPositions[n]):
                    my_NewText[i] = ""

        my_NewText = "".join(my_NewText)

        # printing the text
        return my_NewText, list(zip(myMistakes, myCorrections))
    else:
        return len(myMistakes) == 0
error_check_and_correct("He go to shchool", True)

ModuleNotFoundError: No java install detected. Please install java to use language-tool-python.

In [9]:
def statistics(conn):
    df = pd.read_sql_query("SELECT * FROM caption", conn)
    tgt = 2500
    print("## Target Captions:", tgt)
    print("## Captions:", df.shape[0])
    error_marked = df['is_error'].sum()
    print("## Error Sentence marked:", error_marked)
    print("## Error Sentence marked %:", round(error_marked / df.shape[0] * 100, 3))
    error_found = df['caption'].apply(error_check_and_correct).sum()
    print("## Error Sentence found:", error_found)
    print("## Error Sentence found %:", round(error_found / df.shape[0] * 100, 3))
    print("## Captions %:", df.shape[0]/ tgt * 100)
    print("\n## Caption Count/user:")
    print(df['user'].value_counts(sort=False))
    print("\n## Caption %/user:")
    print(df['user'].value_counts(sort=False) / tgt * 100)
    df['one'] = 1
    gp = df.groupby(by=['file_id', 'caption'])
    tdf = gp.count()['one']
    print("\n## Duplicate:")
    print(tdf[tdf > 1])
    print("\n## Malformated:")
    print(df[df['caption'].apply(lambda x: len(x.split(' ')) < 4)][['file_id', 'caption']])
    
    print("")
statistics(conn)

## Target Captions: 2500
## Captions: 1224
## Error Sentence marked: 15
## Error Sentence marked %: 1.225


ModuleNotFoundError: No java install detected. Please install java to use language-tool-python.

TypeError: statistics() missing 1 required positional argument: 'conn'

In [35]:
df.head()

,id,file_id,caption,user,is_error,is_occluded
0,1,P00001.jpg,"Two knives, one placed on top of other, in a b...",anshul,0,1
1,2,P00001.jpg,A bag with knives.\nTwo shrap knives in a back...,soumen,0,1
2,3,P00001.jpg,Security discovered a concealed knife in the p...,soumen,0,1
3,5,P00004.jpg,Two knives are hidden inside a backpack.,soumen,0,0
4,6,P00005.jpg,Two knives are hidden inside a backpack overla...,soumen,0,0


In [36]:
# User wise caption Count
df['file_id'].unique().shape

(432,)

In [37]:
# User wise caption Count
df['user'].value_counts()

soumen     762
sriram     405
murugan     35
tejas       15
anshul       7
Name: user, dtype: int64

In [38]:
# User wise caption Percentage
df['user'].value_counts() / 2500 * 100

soumen     30.48
sriram     16.20
murugan     1.40
tejas       0.60
anshul      0.28
Name: user, dtype: float64

```
soumen     30.48
sriram     16.20
murugan     1.40
tejas       0.60
anshul      0.28
```

```
soumen     16.68
sriram     13.24
murugan     0.68
tejas       0.60
anshul      0.28
```

In [39]:
# overall progress in percentage
df['user'].value_counts().sum() / 2500 * 100

48.96

In [31]:
user = "soumen"

In [32]:
df.shape

(1223, 6)

In [219]:
dfs = df[df['user'] == user]
dfs.shape

(417, 6)

In [84]:
# 1 captions for soumen
fileids = (dfs['file_id'].value_counts() < 2).index
fileids = fileids[(dfs['file_id'].value_counts() <2).values]
fileids.shape

(182,)

In [85]:
# 0 captions for soumen
df[df['user'] != user]['file_id']

0      P00001.jpg
33     P00001.jpg
51     P00004.jpg
52     P00001.jpg
53     P00005.jpg
          ...    
782    P02382.jpg
783    P02385.jpg
784    P02389.jpg
785    P02393.jpg
786    P02407.jpg
Name: file_id, Length: 370, dtype: object

In [87]:
dfs[dfs['file_id'].isin(fileids)].to_csv("caption_1_{}.csv".format(user), index=False)

# Read captions for `user=soumen`

In [89]:
user = 'soumen'
dfs = df[df['user'] == user]
dfs.shape

(417, 6)

In [138]:
len(lines)

69

In [100]:
# !pip install nltk
import nltk
from nltk.corpus import wordnet
# nltk.download('punkt')
# nltk.download('wordnet')
def get_synonyms(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return synonyms

def paraphrase_text(text):
    paraphrased_text = []
    words = nltk.word_tokenize(text)
    for word in words:
        synonyms = get_synonyms(word)
        if synonyms:
            paraphrased_text.append(synonyms[0])
        else:
            paraphrased_text.append(word)
    return ' '.join(paraphrased_text)

# Example usage
original_text = "A trolley bag contains a sharp knife."
paraphrased_text = paraphrase_text(original_text)

print("Original text:", original_text)
print("Paraphrased text:", paraphrased_text)


Original text: A trolley bag contains a sharp knife.
Paraphrased text: angstrom streetcar bag incorporate angstrom sharp knife .


In [104]:
# !pip install transformers
from transformers import AutoModelWithLMHead, AutoTokenizer
model_name = "t5-base"  # Pre-trained model name
model = AutoModelWithLMHead.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
def paraphrase_text(text):

    # Tokenize the input text
    input_ids = tokenizer.encode(text, return_tensors="pt")

    # Generate paraphrased text
    paraphrased_ids = model.generate(input_ids, max_length=50, num_return_sequences=1, early_stopping=True)
    paraphrased_text = tokenizer.decode(paraphrased_ids[0], skip_special_tokens=True)

    return paraphrased_text

# Example usage
original_text = "a gun, an Wi-Fi router and a sharp object with a water bottle inside a trolly bag."
paraphrased_text = paraphrase_text(original_text)

print("Original text:", original_text)
print("Paraphrased text:", paraphrased_text)


Original text: a gun, an Wi-Fi router and a sharp object with a water bottle inside a trolly bag.
Paraphrased text: a gun, a Wi-Fi router and a sharp object with a water bottle inside a trolly bag.


In [143]:
s = ""
ctr = 0
i = 0
for x in dfs.caption.unique():
    if len(s) + len(x) > 1000:
        print(s, "\n", ctr, "\n\n\n")
        s = ""
        ctr = 0
    ctr += 1
    s = "{}{}.\n".format(s, x.strip(".").strip())
    s = s.replace("..", ".")

A bag with knives.
Two shrap knives in a backpack.
Security discovered a concealed knife in the passenger's bag.
Two knives are hidden inside a backpack.
Two knives are hidden inside a backpack overlapping each other.
A bag contains a sharp knife at the bottom with a few electronics items.
Two knives are hidden inside a luggage scattered around.
Two knives are hidden inside a luggage.
Two knives are hidden inside a luggage scattered around.
A trolly is containing multiple weapons.
a luggage is hiding a few knives.
A luggage is containing deadly knives and two guns.
Two knives are scattered in a bag.
Four threat items present in a trolly bag, out of which two guns and two knives.
A bag with sharp knives.
Two shrap knives in a backpack.
Two sharp knives in a backpack.
Two sharp knives and a few water bottles in a backpack.
Two sharp knives and a few water bottles in a backpack.
Two sharp knives and a few other items in a backpack.
 
 19 



Four threat items present in a trolly bag, out 

In [198]:
lines = """A bag containing knives.
Two sharp knives are inside a backpack.
Security discovered a hidden knife in the passenger's bag.
Two knives are concealed within a backpack.
Two knives are covertly placed inside a backpack, overlapping each other.
At the bottom of the bag, there is a sharp knife along with a few electronics items.
Two knives are hidden within a scattered luggage.
Two knives are concealed inside a luggage.
Two knives are secretly tucked away inside a luggage that is scattered around.
The trolley contains multiple weapons.
A luggage conceals a few knives.
A luggage contains deadly knives and two guns.
Two knives are scattered in a bag.
The trolley bag contains four threatening items, including two guns and two knives.
A bag with sharp knives.
Two sharp knives are in a backpack.
Two sharp knives are present in a backpack.
A backpack contains two sharp knives and a few water bottles.
Two sharp knives and several water bottles are inside a backpack.
Inside a backpack, there are two sharp knives along with a few other items.
A trolley bag contains four dangerous items, comprising two guns and two knives.
Inside a backpack, two knives are concealed.
In a bag, multiple items including knives and guns are overlapping and hidden.
Guns, sharp knives, and other items are concealed within a bag.
Several items, including two overlapping knives, guns, and others, are hidden inside a bag.
Two guns are stashed within a trolley bag.
At the bottom of a trolley bag, two guns and several knives are concealed.
A trolley bag conceals two guns and two knives at its bottom.
Inside a luggage, a gun is hidden along with several other metal components.
In a luggage, a gun is concealed along with other metal components, a laptop, and an umbrella.
An umbrella is present in a bag filled with guns and other metal items.
Guns and other metal items are packed inside a bag, along with an umbrella.
A trolley bag is fully loaded with guns.
A handbag contains a gun, a Wi-Fi router, a sharp object, and a water bottle.
Inside a trolley bag, there is a gun, a Wi-Fi router, a sharp object, and a water bottle.
A handbag conceals a toolbox and a gun.
Inside a handbag, there is a toolbox hidden along with two guns.
A handbag is packed with a laptop and concealed guns.
An almost empty trolley bag discreetly hides a covered gun.
A gun is hidden inside an almost empty trolley bag.
A gun is concealed within a suitcase.
Two guns are positioned in the middle of a handbag, overlapping each other.
A handbag contains five guns in its middle portion.
A trolley bag holds one gun, an umbrella, and a few other items.
Inside a handbag, there is a flask and a gun.
A handbag is equipped with an umbrella, a water bottle, and a concealed gun.
Inside a trolley bag, there is an umbrella, a water bottle, and a concealed gun.
A handbag is packed with a laptop and a gun.
A handbag is filled with a laptop, a knife, and concealed items.
A trolley bag is filled with knives, two guns, and a highly occluded laptop.
A trolley bag is full of sharp knives, guns, and various other items.
A trolley bag contains a sharp knife hidden inside.
A backpack contains a kitchen knife and two guns.
A trolley bag contains three knives, including one kitchen knife and two guns.
A backpack conceals a kitchen knife and two guns.
A trolley bag includes a sharp knife.
A trolley bag contains a sharp knife.
A laptop bag holds two guns and a knife.
An umbrella and a knife are concealed inside a laptop bag.
A laptop bag conceals a knife and an umbrella.
A laptop bag contains concealed knives and guns.
A laptop bag conceals a knife.
A laptop bag contains concealed knives.
A laptop handbag conceals hidden guns.
A trolley bag with two guns contains a sharp knife.
A trolley bag with two guns holds two sharp knives.
A carry-on luggage contains a knife.
A toolbox and a gun are concealed inside a trolley bag.
A toolbox and a gun are hidden inside a backpack.
A backpack contains a kitchen knife and two firearms.
A sharp knife is placed inside a trolley bag.
Inside a trolley bag, there is a sharp knife.
A trolley bag conceals a sharp knife.
A laptop bag holds two guns and a knife.
A laptop bag contains a knife and an umbrella.
A knife and an umbrella are concealed inside a laptop bag.
A laptop bag contains concealed knives and guns.
A laptop bag conceals a hidden knife.
A laptop bag contains concealed knives.
A laptop handbag conceals hidden guns.
Inside a trolley bag with two guns, there is a sharp knife.
Inside a trolley bag with two guns, two sharp knives are placed.
A carry-on luggage contains a knife.
A toolbox and a gun are hidden in a trolley bag.
A toolbox and a gun are concealed inside a backpack.
A backpack conceals a toolbox and guns.
A small backpack contains a dangerous knife.
A tray contains a knife and several other prohibited items.
A backpack is filled with guns.
A trolley bag contains a firearm.
A trolley bag contains three guns and a laptop, heavily concealed.
A handbag is packed with guns.
Two guns are inside a handbag.
There is a gun on top of a handbag.
Two guns are concealed within a handbag.
Two guns are placed inside a backpack.
Two guns are hidden inside a backpack and a handbag.
Two guns are concealed within a piece of luggage.
A gun is inside a bag.
There is a gun in a bag.
A gun is concealed within a piece of luggage.
A gun is hidden inside a backpack.
There is a gun in a handbag.
A gun is concealed inside a backpack.
A trolley bag contains a gun along with a large metallic object.
At the bottom of a backpack, two guns and one knife are hidden.
Two guns and one knife are concealed at the bottom of a backpack.
Inside a piece of luggage, two knives are hidden.
Several knives are concealed within a piece of luggage.
At the bottom of a trolley bag, two guns and a few knives are hidden.
At the bottom of a trolley bag, two guns and two knives are concealed.
A sharp knife is discreetly hidden inside a trolley bag.
A trolley bag with two guns also contains two sharp knives.
Two sharp knives are placed inside a trolley bag with two guns.
A trolley bag holds a sharp knife alongside two guns.
Two sharp knives are positioned inside a trolley bag along with two guns.
There is a firearm concealed within a trolley bag.
A handbag contains both a gun and a laptop.
A firearm is present inside a bag on wheels.
An enclosed trolley bag houses a gun.
A gun is concealed within a trolley bag alongside a large metallic object.
A gun is hidden within a trolley bag along with a laptop.
A trolley bag contains a gun along with various other metal and electronic items.
A gun is concealed within a trolley bag along with other metal and electronic items, and it is highly obscured.""".split("\n")

captions = """A bag with knives.
Two shrap knives in a backpack.
Security discovered a concealed knife in the passenger's bag.
Two knives are hidden inside a backpack.
Two knives are hidden inside a backpack overlapping each other.
A bag contains a sharp knife at the bottom with a few electronics items.
Two knives are hidden inside a luggage scattered around.
Two knives are hidden inside a luggage.
Two knives are hidden inside a luggage scattered around.
A trolly is containing multiple weapons.
a luggage is hiding a few knives.
A luggage is containing deadly knives and two guns.
Two knives are scattered in a bag.
Four threat items present in a trolly bag, out of which two guns and two knives.
A bag with sharp knives.
Two shrap knives in a backpack.
Two sharp knives in a backpack.
Two sharp knives and a few water bottles in a backpack.
Two sharp knives and a few water bottles in a backpack.
Two sharp knives and a few other items in a backpack.
Four threat items present in a trolly bag, out of which two guns and two knives.
Two knives are hidden inside a backpack.
Two overlapped knives, guns and other items are occluded in a baggage.
Sharp knives, guns and other items are occluded in a baggage.
Two overlapped knives, guns and other items are occluded in a baggage.
Two guns in a trolly bag.
Two guns a few knives are hidden in the bottom of a trolly bag.
Two guns and two knives are hidden in the bottom of a trolly bag.
A gun is hidden with a few other metal components in a luggage.
A gun is hidden with a few other metal components in a luggage.
A gun is hidden with a few other metal components, laptop and umbrella in a luggage.
A gun is hidden with a few other metal components, laptop and umbrella in a luggage.
A bag is full of guns and other metal items with an umbrella.
A bag is full of guns and other metal items with an umbrella.
A trolly bag is full of guns.
A handbag contains a gun and an Wi-Fi router and a sharp object with a water bottle inside.
A handbag contains a gun and an Wi-Fi router and a sharp object with a water bottle inside.
a gun, an Wi-Fi router and a sharp object with a water bottle inside a trolly bag.
A toolbox and a gun is hidden in a hand-bag.
A toolbox and a gun is hidden in a hand-bag.
A toolbox and two guns are hidden in a hand-bag.
A toolbox and two guns are hidden in a trolly-bag.
A handbag packed with Laptop and hiding two guns.
A handbag packed with Laptop and hiding two guns.
An almost empty trolly bag is hiding a gun covered with clothes.
An almost empty trolly bag is hiding a gun.
A suitcase is hiding a gun.
Two guns are placed in the middle of a handbag and overlapping each other.
Two guns are placed in the middle of a handbag and overlapping each other.
Two guns are placed in the middle of a handbag.
Five guns are present in a handbg.
Five guns are placed in the middle of a handbag and overlapping each other.
A trolly bag containing one gun, an umbrella and a few other items.
A trolly bag containing one gun, an umbrella and a few other items.
One flask and a gun inside a handbug.
One flask and a gun inside a handbag.
A handbag is packed with an umbrella, water bottle and a gun.
A handbag is packed with an umbrella, water bottle and a gun.
A handbag is packed with an umbrella, water bottle and an occluded gun.
A trolly bag is packed with an umbrella, water bottle and an occluded gun.
A trolly bag is packed with an umbrella, water bottle and an occluded gun.
A handbag is packed with Laptop and gun.
A handbag is packdd with Laptop and gun.
A handbag is packed with Laptop and knife and occluded.
A trolly bag is full of knives and two guns and a laptop highly occluded.
A trolly-bag full of sharp knives, guns and a few other items.
A sharp knife is hidden inside a trolly-bag.
A backpack contains a kitchen knife and two guns.
A trolly bag contains three knives, including one kitchen knife and two guns.
A backpack contains a kitchen knife and two guns.
Sharo knife is placed in a trolly bag.
A trolly bag contains a sharp knife.
A trolly bag contains a sharp knife.
A Laptop bag contains two guns and a knife.
A Laptop bag contains a knife and umbrella.
A Laptop bag contains occluded a knife and umbrella.
A Laptop bag contains occluded a knives and guns.
A Laptop bag contains occluded knife.
A Laptop bag contains occluded knives.
A Laptop handbag contains occluded guns.
Sharp knife is placed in a trolly bag with two guns.
Two sharp knife is placed in a trolly bag with two guns.
A carry-on luggage contains knife.
A toolbox and a gun is hidden in a trolly-bag.
A toolbox and a gun is hidden in a backpack.
A toolbox and guns are hidden in a backpack.
A small backpack contains a deadly knife.
A tray contains a knife and few other prohibited items.
A backpack is full of guns.
A trolly bag contains a gun.
A trolly three guns and a laptop highly occluded.
A handbag is full of guns.
Two guns in a hand bag.
A gun on top of a hand bag.
Two guns in a hand bag.
Two guns in a backpack.
Two guns in a backpack.
Two guns in a backpack and a handbag.
Two guns in a luggage.
A gun in a bag.
A gun in a bag.
A gun in a luggage.
A gun in a backpack.
1745.
A gun in a handbag.
A gun in a backpack.
A gun in a trolly bag with a big meta object.
Two guns and one knife is hidden in the bottom of a backpack.
Two guns and one knife is hidden in the bottom of a backpack.
Two knives are hidden inside a luggage.
a luggage is hiding a few knives.
Two guns a few knives are hidden in the bottom of a trolly bag.
Two guns and two knives are hidden in the bottom of a trolly bag.
A sharp knife is hidden inside a trolly-bag.
Sharp knife is placed in a trolly bag with two guns.
Two sharp knife is placed in a trolly bag with two guns.
A gun in  a trolly bag
A gun in a handbag with a laptop
A gun in a troll bag
A gun in a trolly bag
A gun in a trolly bag with a big metal object
A gun in a trolly bag with a laptop
A gun in a trolly bag with other metal and electronics items
A gun in a trolly bag with other metal and electronics items and highly occluded""".split("\n")

caption1_map = defaultdict(lambda: [])
remain = []
for i in range(len(captions)):
    caption1_map[captions[i].strip().strip('.').strip()].append(lines[i] if i < len(lines) else "")
    if i >= len(lines):
        remain.append(captions[i])
for x in remain:
    print(x)
caption1_map

defaultdict(<function __main__.<lambda>()>,
            {'A bag with knives': ['A bag containing knives.'],
             'Two shrap knives in a backpack': ['Two sharp knives are inside a backpack.',
              'Two sharp knives are in a backpack.'],
             "Security discovered a concealed knife in the passenger's bag": ["Security discovered a hidden knife in the passenger's bag."],
             'Two knives are hidden inside a backpack': ['Two knives are concealed within a backpack.',
              'Inside a backpack, two knives are concealed.'],
             'Two knives are hidden inside a backpack overlapping each other': ['Two knives are covertly placed inside a backpack, overlapping each other.'],
             'A bag contains a sharp knife at the bottom with a few electronics items': ['At the bottom of the bag, there is a sharp knife along with a few electronics items.'],
             'Two knives are hidden inside a luggage scattered around': ['Two knives are hidden within 

In [220]:
import random
not_found = []
def caption_paraphreaser(caption):
    cleaned = "{}".format(caption.strip().strip('.').strip())
    alternative_captions = caption1_map[cleaned]
    if len(alternative_captions) == 0:
        print("NOT AVAILABLE: '{}', '{}'".format(caption, cleaned))
        not_found.append(caption)
        return "Two sharp knives are inside a backpack."
    if len(alternative_captions) == 1:
        return alternative_captions[0]
    idx = random.randint(0, len(alternative_captions)-1)
    return alternative_captions[idx]

In [221]:
dfs['caption_2'] = dfs['caption'].apply(caption_paraphreaser)

NOT AVAILABLE: 'A bag with knives.
Two shrap knives in a backpack.', 'A bag with knives.
Two shrap knives in a backpack'


C:\Users\dndlssardar\AppData\Local\Temp\ipykernel_30892\1797245890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs['caption_2'] = dfs['caption'].apply(caption_paraphreaser)


In [224]:
dfs.to_csv("caption2.csv", index=False)